Najpierw wybierzmy 3 koncepty, które chcemy mieć w naszych danych. W tym celu poszukamy takich konceptów, które występują zarówno w Checkpoints and Assigned Concept Names oraz w datasecie ImageNET100, ponieważ chcemy brać dane z ImageNET i chcemy mieć koncepty wykrywalne przez model.

In [1]:
import json
import pandas as pd

# Wczytaj słownik klas ImageNet100 
imagenet100_concepts_json = json.load(open("Labels.json", "r"))

# Wczytaj plik z nazwami konceptów z repo autorów (Checkpoints and Assigned Concept Names)
concepts_df = pd.read_csv("clip_ViT-L_14concept_names.csv", header=None, names=["neuron_id", "concept"])

# Szukaj pokrywających się słów między konceptami a etykietami ImageNet100
matches = []
for concept in concepts_df["concept"]:
    for wnid, label in imagenet100_concepts_json.items():
        if concept.lower() == label.lower():  
            matches.append((concept, wnid, label))

matches[:10000]


[('pelican', 'n02051845', 'pelican'),
 ('peacock', 'n01806143', 'peacock'),
 ('crane', 'n02012849', 'crane'),
 ('tick', 'n01776313', 'tick'),
 ('flamingo', 'n02007558', 'flamingo'),
 ('crane', 'n02012849', 'crane'),
 ('flamingo', 'n02007558', 'flamingo'),
 ('kite', 'n01608432', 'kite'),
 ('kite', 'n01608432', 'kite'),
 ('crane', 'n02012849', 'crane'),
 ('flamingo', 'n02007558', 'flamingo'),
 ('kite', 'n01608432', 'kite'),
 ('goose', 'n01855672', 'goose'),
 ('drake', 'n01847000', 'drake'),
 ('kite', 'n01608432', 'kite')]

Wybierzmy flamingi, gęsi i pelikany (tematycznie będzie, same zwierzęta).

Tworzymy funkcję do tworzenia zbiorów danych z ImageNET z połową zdjęć z danym konceptem, a połową bez tego konceptu.

In [ ]:
import os
import shutil
import random
from pathlib import Path

def create_concept_binary_dataset(imagenet_root, output_dir, concept_class="n02051845", samples_per_class=1300, concept_name="pelican"):
    """
    Tworzy dataset binarny: z konceptem (1) vs inne (0).
    
    Args:
        imagenet_root: główny folder z ImageNet100, np. "./imagenet100/train"
        output_dir: folder wyjściowy np. ./binary_dataset_pelican
        concept_class: kod klasy konceptu w ImageNet100, np. "n02051845" (pelikan)
        samples_per_class: liczba przykładów w każdej klasie, domyślnie 1300 ponieważ tyle zdjęć jest w ImageNet100 w zbiorze treningowym dla każdego konceptu
        concept_name: nazwa konceptu, domyślnie "pelican"
    """

    concept_dir = Path(imagenet_root) / concept_class
    all_classes = [c for c in os.listdir(imagenet_root) if os.path.isdir(os.path.join(imagenet_root, c))]
    non_concept_classes = [c for c in all_classes if c != concept_class]

    
    output_1 = Path(output_dir) / "1_{concept_name}"
    output_0 = Path(output_dir) / "0_other"
    output_1.mkdir(parents=True, exist_ok=True)
    output_0.mkdir(parents=True, exist_ok=True)

    # Wybierz obrazy z klasy konceptu (np. pelikan)
    concept_images = list(concept_dir.glob("*.JPEG"))[:samples_per_class]
    for img in concept_images:
        shutil.copy(img, output_1 / img.name)

    # Wybierz losowo obrazy z innych klas
    other_images = []
    for cls in non_concept_classes:
        class_dir = Path(imagenet_root) / cls
        imgs = list(class_dir.glob("*.JPEG"))
        other_images.extend(imgs)

    random.shuffle(other_images)
    for img in other_images[:samples_per_class]:
        shutil.copy(img, output_0 / img.name)

    print(f"Dataset utworzony: {samples_per_class} z konceptem + {samples_per_class} bez {concept_name}.")
    print(f"Zapisano do: {output_dir}")


Tworzymy folder z połową zdjęć z pelikanami i połową bez.

In [ ]:
create_concept_binary_dataset(
    imagenet_root="./ImageNet100/train",
    output_dir="./binary_dataset_pelican",
    concept_class="n02051845"
)

Tworzymy folder z flamingami i bez.

In [ ]:
create_concept_binary_dataset(
    imagenet_root="./ImageNet100/train",
    output_dir="./binary_dataset_flamingo",
    concept_class="n02007558"
)

Teraz z gęsiami i bez.

In [ ]:
create_concept_binary_dataset(
    imagenet_root="./ImageNet100/train",
    output_dir="./binary_dataset_flamingo",
    concept_class="n01855672"
)